In [291]:
from itertools import combinations
from functools import reduce
import numpy as np

import IPython.display as dp

In [409]:
def render_html(text):
    return dp.HTML("<style>.nowrap{white-space:pre;font-family:monospace}</style><span class='nowrap' >" + text + "</span>")

class HammingException(Exception):
    messages = {
        1: 'Not all codes of the same length',
        2: 'Invalid number of bits requested',
        3: 'Can\'t encode an empty bitstring',
    }
    def __init__(self, selector):
        Exception.__init__(self, HammingException.messages[selector])
    
def combos(iterable):
    if not len(set([len(i) for i in iterable])) == 1:
        raise HammingException(1) 
    return list(combinations(iterable, 2))

class HammingDistance():
    def __init__(self, iterable):
        self.codes = iterable
        self._combos = combos(iterable)
        self.min_distance = self.get_min_distance()
        
    def get_min_distance(self, pairs=None):
        c = self._combos if pairs is None else combos(pairs)
        self.visualize_distances = [HammingDistance.compare_pair((h1, h2)) for h1, h2 in c]
        self._distances = [c[0] for c in self.visualize_distances]
        self.Dmin = np.nanmin([(c if c != 0 else np.nan) for c in self._distances]).astype(int)
        
    def compare_pair(pair):
        l0, l1, l2 = [], [], []
        for i, j in zip(pair[0], pair[1]):
            l0.append(1 if i != j else 0)
            l1.append(i)
            l2.append(j)
        return sum(l0), l1, l2

class HammingEncoder():
    def __init__(self, data=None, data_bits=None):
        
        if data is not None:
            data_bits = len(data)
        elif data_bits is None:
            raise HammingException(3)
            
        self.data_bits = data_bits
        self.redundant_bits = HammingEncoder.calculate_redundant_bits(self.data_bits)
        self._redundant_positions = [2**i for i in range(self.redundant_bits)]
        self.encoding = {
            i + 1: HammingBit(
                              i + 1,
                              ' ',
                              (True if i + 1 in self._redundant_positions else False)
                             )
            for i in range(self.data_bits + self.redundant_bits)
        }
        if data is not None:
            self.fill(data)
        
    def fill(self, data):
        data_range = [i for i in self.encoding.keys() if self.encoding[i].is_redundant == False]
        for i, j in enumerate(data_range):
            self.encoding[j].value = data[i]
        
    def render_string(self, html=False):
        breaker = '\n' if not html else '<br>'
        max_length = max([len(str(i)) for i in self.encoding.keys()])
        chars = [HammingEncoder.rendered_char(bit, max_length) for bit in list(self.encoding.values())[::-1]]
        return breaker.join(["".join([c[i] for c in chars]) for i in range(5)])
    
    def get_as_html(self):
        return render_html(self.render_string(html=True))
    
    def __print__(self):
        print(self.render_string())
        
    def rendered_char(bit, max_length=3):
        
        val_dif = max_length - len(str(bit.value))
        pdif = max_length - len(str(bit.place))
        
        if bit.is_redundant:
            return [
                f'-{"-" * max_length}-',
                f'|{" " * val_dif}{bit.value}|',
                f'|{"-" * max_length}|',
                f'|{" " * pdif}{bit.place}|',
                f'-{"-" * max_length}-',
            ]
        else:
            return [
                f' {" " * max_length} ',
                f' {" " * val_dif}{bit.value} ',
                f' {"-" * max_length} ',
                f' {" " * pdif}{bit.place} ',
                f' {" " * max_length} ',
            ]
        
    def calculate_redundant_bits(data_bits):
        m = data_bits + 1
        for r in range(1000):
            if m + r <= 2**r:
                return r
        raise HammingException(2)
    


class HammingBit():
    def __init__(self, place, value, is_redundant):
        self.place = place
        self.value = value
        self.is_redundant = is_redundant
    def __str__(self):
        return f'HammingBit(value = {self.value}, is_redundant = {self.is_redundant})'
        

In [410]:
codes = ['100001101111', '100000001111', '100000001110', '100001101111']

In [411]:
hd = HammingDistance(['111', '111'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: All-NaN axis encountered


In [412]:
hd.Dmin

-2147483648

In [413]:
hd.get_min_distance(['10001', '01110'])

In [423]:
he = HammingEncoder(data_bits=100)

In [424]:
he.redundant_bits

7

In [425]:
print(he.encoding[3])

HammingBit(value =  , is_redundant = False)


In [426]:
he._redundant_positions

[1, 2, 4, 8, 16, 32, 64]

In [427]:
he.__print__()

                                                                                                                                                                                                                       -----                                                                                                                                                           -----                                                                           -----                                   -----               -----     ----------
                                                                                                                                                                                                                       |   |                                                                                                                                                           |   |                                                                           |   |    

In [428]:
out = he.get_as_html()

In [429]:
type(out)

IPython.core.display.HTML